<p style="align: center;"><img align=center src="https://drive.google.com/uc?export=view&id=1I8kDikouqpH4hf7JBiSYAeNT2IO52T-T" width=600 height=480/></p>
<h3 style="text-align: center;"><b>Школа глубокого обучения ФПМИ МФТИ</b></h3>

<h3 style="text-align: center;"><b>Домашнее задание. Generative adversarial networks</b></h3>



В этом домашнем задании вы обучите GAN генерировать лица людей и посмотрите на то, как можно оценивать качество генерации

In [1]:
import os
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
import torch
import torch.nn as nn
import cv2
from tqdm.notebook import tqdm
from torchvision.utils import save_image
from torchvision.utils import make_grid
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(style='darkgrid', font_scale=1.2)

In [4]:
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision import datasets

## Часть 1. Подготовка данных (0.5 балла)

В качестве обучающей выборки возьмем часть датасета [Flickr Faces](https://github.com/NVlabs/ffhq-dataset), который содержит изображения лиц людей в высоком разрешении (1024х1024). Оригинальный датасет очень большой, поэтому мы возьмем его часть. Скачать датасет можно [здесь](https://www.kaggle.com/datasets/tommykamaz/faces-dataset-small?resource=download-directory) и  [здесь](https://drive.google.com/file/d/1inyvLrN5wKBGCxQ4znMKBc64uL4uP_2x/view?usp=drive_link)

Давайте загрузим наши изображения. Напишите функцию, которая строит DataLoader для изображений, при этом меняя их размер до нужного значения (размер 1024 слишком большой, поэтому мы рекомендуем взять размер 128 либо немного больше)

In [ ]:
def get_dataloader(image_size, batch_size):
  """
  Builds dataloader for training data.
  Use tt.Compose and tt.Resize for transformations
  :param image_size: height and wdith of the image
  :param batch_size: batch_size of the dataloader
  :returns: DataLoader object
  """
  transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    tt.CenterCrop(image_size),
    transforms.ToTensor(),
    tt.Normalize([0.5]*3, [0.5]*3)
  ])

  dataset = datasets.ImageFolder(root="", transform=transform)
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
  return dataloader

In [ ]:
image_size = 128
batch_size = 32
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataloader = get_dataloader(image_size, batch_size)

Выведем картинку из импортированного датасета

In [ ]:
images, _ = next(iter(dataloader))
images = images.to(device)
print(images.shape)

## Часть 2. Построение и обучение модели (2 балла)

Сконструируйте генератор и дискриминатор. Помните, что:
* дискриминатор принимает на вход изображение (тензор размера `3 x image_size x image_size`) и выдает вероятность того, что изображение настоящее (тензор размера 1)

* генератор принимает на вход тензор шумов размера `latent_size x 1 x 1` и генерирует изображение размера `3 x image_size x image_size`

In [ ]:
discriminator = nn.Sequential(
    # in: 3 x 64 x 64

    nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 64 x 32 x 32

    nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 128 x 16 x 16

    nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 256 x 8 x 8

    nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(512),
    nn.LeakyReLU(0.2, inplace=True),
    # out: 512 x 4 x 4

    nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=0, bias=False),
    # out: 1 x 1 x 1

    nn.Flatten())

In [ ]:
latent_size = 128

generator = nn.Sequential(
    # in: latent_size x 1 x 1

    nn.ConvTranspose2d(latent_size, 512, kernel_size=4, stride=1, padding=0, bias=False),
    nn.BatchNorm2d(512),
    nn.ReLU(True),
    # out: 512 x 4 x 4

    nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(256),
    nn.ReLU(True),
    # out: 256 x 8 x 8

    nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(128),
    nn.ReLU(True),
    # out: 128 x 16 x 16

    nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1, bias=False),
    nn.BatchNorm2d(64),
    nn.ReLU(True),
    # out: 64 x 32 x 32

    nn.ConvTranspose2d(64, 3, kernel_size=4, stride=2, padding=1, bias=False),
    nn.Tanh()
    # out: 3 x 64 x 64
)

Перейдем теперь к обучению нашего GANа. Алгоритм обучения следующий:
1. Учим дискриминатор:
  * берем реальные изображения и присваиваем им метку 1
  * генерируем изображения генератором и присваиваем им метку 0
  * обучаем классификатор на два класса

2. Учим генератор:
  * генерируем изображения генератором и присваиваем им метку 0
  * предсказываем дискриминаторором, реальное это изображение или нет


В качестве функции потерь берем бинарную кросс-энтропию

In [ ]:
lr = 0.0001

model = {
    "discriminator": discriminator,
    "generator": generator
}

# Функции потерь не нужны — считаем вручную:
# Для дискриминатора:
loss_d = -torch.mean(real_preds) + torch.mean(fake_preds)

# Для генератора:
loss_g = -torch.mean(fake_preds)

In [ ]:
def fit(model, criterion, epochs, lr, start_idx=1):
    model["discriminator"].train()
    model["generator"].train()
    torch.cuda.empty_cache()

    # Losses & scores
    losses_g = []
    losses_d = []
    real_scores = []
    fake_scores = []

    # Create optimizers
    optimizer = {
        "discriminator": torch.optim.Adam(model["discriminator"].parameters(),
                                          lr=lr, betas=(0.5, 0.999)),
        "generator": torch.optim.Adam(model["generator"].parameters(),
                                      lr=lr, betas=(0.5, 0.999))
    }

    n_critic = 5

    for epoch in range(epochs):
        loss_d_per_epoch = []
        loss_g_per_epoch = []
        real_score_per_epoch = []
        fake_score_per_epoch = []
        for i, (real_images, _) in enumerate(tqdm(train_dl)):
            # Train discriminator
            # Clear discriminator gradients
            optimizer["discriminator"].zero_grad()

            # Pass real images through discriminator
            real_preds = model["discriminator"](real_images)
            real_targets = torch.ones(real_images.size(0), 1, device=device)
            loss_d = -real_preds.mean() + fake_preds.mean(
            cur_real_score = torch.mean(real_preds).item()

            # Generate fake images
            latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
            fake_images = model["generator"](latent)

            # Pass fake images through discriminator
            fake_targets = torch.zeros(fake_images.size(0), 1, device=device)
            fake_preds = model["discriminator"](fake_images)
            fake_loss = criterion["discriminator"](fake_preds, fake_targets)
            cur_fake_score = torch.mean(fake_preds).item()

            real_score_per_epoch.append(cur_real_score)
            fake_score_per_epoch.append(cur_fake_score)

            # Update discriminator weights
            loss_d = real_loss + fake_loss
            loss_d.backward()
            optimizer["discriminator"].step()
            loss_d_per_epoch.append(loss_d.item())

            for i, (real_images, _) in enumerate(tqdm(train_dl)):
              # Train generator
              # Clear generator gradients
              optimizer["generator"].zero_grad()

              # Generate fake images
              latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
              fake_images = model["generator"](latent)

              # Try to fool the discriminator
              preds = model["discriminator"](fake_images)
              targets = torch.ones(batch_size, 1, device=device)
              loss_g = -preds.mean()

              # Update generator weights
              loss_g.backward()
              optimizer["generator"].step()
              loss_g_per_epoch.append(loss_g.item())

        # Record losses & scores
        losses_g.append(np.mean(loss_g_per_epoch))
        losses_d.append(np.mean(loss_d_per_epoch))
        real_scores.append(np.mean(real_score_per_epoch))
        fake_scores.append(np.mean(fake_score_per_epoch))

        # Log losses & scores (last batch)
        print("Epoch [{}/{}], loss_g: {:.4f}, loss_d: {:.4f}, real_score: {:.4f}, fake_score: {:.4f}".format(
            epoch+1, epochs,
            losses_g[-1], losses_d[-1], real_scores[-1], fake_scores[-1]))

        # Save generated images
        if epoch == epochs - 1:
          save_samples(epoch+start_idx, fixed_latent, show=False)

    return losses_g, losses_d, real_scores, fake_scores

In [ ]:
def fit(model, epochs, lr, start_idx=1, lambda_gp=10):
    model["discriminator"].train()
    model["generator"].train()
    torch.cuda.empty_cache()

    optimizer = {
        "discriminator": torch.optim.Adam(model["discriminator"].parameters(), lr=lr, betas=(0.5, 0.999)),
        "generator": torch.optim.Adam(model["generator"].parameters(), lr=lr, betas=(0.5, 0.999))
    }

    n_critic = 5
    losses_g, losses_d, real_scores, fake_scores = [], [], [], []

    for epoch in range(epochs):
        loss_d_per_epoch, loss_g_per_epoch = [], []
        real_score_per_epoch, fake_score_per_epoch = [], []

        for i, (real_images, _) in enumerate(tqdm(train_dl)):
            real_images = real_images.to(device)

            # ========== Обновляем дискриминатор n_critic раз ==========
            for _ in range(n_critic):
                optimizer["discriminator"].zero_grad()

                # Генерируем фейковые изображения
                latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
                fake_images = model["generator"](latent).detach()

                real_preds = model["discriminator"](real_images)
                fake_preds = model["discriminator"](fake_images)

                # loss = -E[real] + E[fake]
                loss_d = -real_preds.mean() + fake_preds.mean()

                # [опционально] добавить gradient penalty
                # gp = gradient_penalty(...)
                # loss_d += lambda_gp * gp

                loss_d.backward()
                optimizer["discriminator"].step()

                real_score_per_epoch.append(real_preds.mean().item())
                fake_score_per_epoch.append(fake_preds.mean().item())
                loss_d_per_epoch.append(loss_d.item())

            # ========== Обновляем генератор ==========
            optimizer["generator"].zero_grad()

            latent = torch.randn(batch_size, latent_size, 1, 1, device=device)
            fake_images = model["generator"](latent)
            preds = model["discriminator"](fake_images)
            loss_g = -preds.mean()

            loss_g.backward()
            optimizer["generator"].step()

            loss_g_per_epoch.append(loss_g.item())

        # === Логирование ===
        losses_g.append(np.mean(loss_g_per_epoch))
        losses_d.append(np.mean(loss_d_per_epoch))
        real_scores.append(np.mean(real_score_per_epoch))
        fake_scores.append(np.mean(fake_score_per_epoch))

        print(f"Epoch [{epoch+1}/{epochs}], "
              f"loss_g: {losses_g[-1]:.4f}, "
              f"loss_d: {losses_d[-1]:.4f}, "
              f"real_score: {real_scores[-1]:.4f}, "
              f"fake_score: {fake_scores[-1]:.4f}")

        if epoch == epochs - 1:
            save_samples(epoch + start_idx, fixed_latent, show=False)

    return losses_g, losses_d, real_scores, fake_scores


Постройте графики лосса для генератора и дискриминатора. Что вы можете сказать про эти графики?

In [ ]:
history = fit(model, criterion, epochs, lr)

In [ ]:
losses_g, losses_d, real_scores, fake_scores = history

In [ ]:
generated_img = cv2.imread(f'./generated/generated-images-00{epochs}.png')
generated_img = generated_img[:, :, [2, 1, 0]]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_xticks([]); ax.set_yticks([])
ax.imshow(generated_img)

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(losses_d, '-')
plt.plot(losses_g, '-')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['Discriminator', 'Generator'])
plt.title('Losses');

In [ ]:
plt.figure(figsize=(15, 6))

plt.plot(real_scores, '-')
plt.plot(fake_scores, '-')
plt.xlabel('epoch')
plt.ylabel('score')
plt.legend(['Real', 'Fake'])
plt.title('Scores');

## Часть 3. Генерация изображений

Теперь давайте оценим качество получившихся изображений. Напишите функцию, которая выводит изображения, сгенерированные нашим генератором

In [ ]:
n_images = 4

fixed_latent = torch.randn(n_images, latent_size, 1, 1, device=device)
fake_images = model["generator"](fixed_latent)

In [ ]:
def show_images(generated):

  pass

Как вам качество получившихся изображений?

## Часть 4. Leave-one-out-1-NN classifier accuracy (6 баллов)

### 4.1. Подсчет accuracy (1.5 балл)

Не всегда бывает удобно оценивать качество сгенерированных картинок глазами. В качестве альтернативы вам предлагается реализовать следующий подход:
  * Сгенерировать столько же фейковых изображений, сколько есть настоящих в обучающей выборке. Присвоить фейковым метку класса 0, настоящим – 1.
  * Построить leave-one-out оценку: обучить 1NN Classifier (`sklearn.neighbors.KNeighborsClassifier(n_neighbors=1)`) предсказывать класс на всех объектах, кроме одного, проверить качество (accuracy) на оставшемся объекте. В этом вам поможет `sklearn.model_selection.LeaveOneOut`

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import LeaveOneOut
import torch
from torchvision import models, transforms
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Шаг 1: Подготовка моделей и трансформаций
feature_extractor = models.resnet18(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(feature_extractor.children())[:-1])  # удаляем FC-слой
feature_extractor.eval().to(device)

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Шаг 2: Сбор фейковых и реальных изображений
real_images = []  # тензоры реальных изображений (N, 3, H, W)
fake_images = []  # тензоры фейковых изображений (N, 3, H, W)

# Предположим, что у тебя уже есть batches:
# real_images = next(iter(train_dl))[0][:N]  ← возьми N реальных
# fake_images = generator(torch.randn(N, latent_size, 1, 1).to(device)).detach().cpu()

# Шаг 3: Преобразование + получение признаков
def extract_features(images_tensor):
    features = []
    with torch.no_grad():
        for img in tqdm(images_tensor):
            img = transform(img).unsqueeze(0).to(device)
            feat = feature_extractor(img).squeeze().cpu().numpy()
            features.append(feat)
    return np.stack(features)

# Реальные и фейковые
X_real = extract_features(real_images)
X_fake = extract_features(fake_images)

X_all = np.vstack([X_real, X_fake])
y_all = np.array([1]*len(X_real) + [0]*len(X_fake))

# Шаг 4: Leave-One-Out + 1NN
loo = LeaveOneOut()
model = KNeighborsClassifier(n_neighbors=1)

correct = 0
total = len(X_all)

for train_index, test_index in tqdm(loo.split(X_all)):
    X_train, X_test = X_all[train_index], X_all[test_index]
    y_train, y_test = y_all[train_index], y_all[test_index]

    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    correct += (pred == y_test)

accuracy = correct / total
print(f"Leave-One-Out 1NN Accuracy: {accuracy:.4f}")


Что вы можете сказать о получившемся результате? Какой accuracy мы хотели бы получить и почему?

### 4.2. Визуализация распределений (1 балл)

Давайте посмотрим на то, насколько похожи распределения настоящих и фейковых изображений. Для этого воспользуйтесь методом, снижающим размерность (к примеру, TSNE) и изобразите на графике разным цветом точки, соответствующие реальным и сгенерированным изображенияи

In [ ]:
import torch
import numpy as np
from torchvision import models, transforms
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tqdm import tqdm

# 1. Готовим feature extractor (без FC-слоя)
resnet = models.resnet18(pretrained=True)
feature_extractor = torch.nn.Sequential(*list(resnet.children())[:-1])  # (512,)
feature_extractor.eval().to(device)

# 2. Преобразования (Resize + Normalize под ImageNet)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# 3. Извлекаем фичи из изображений
def extract_features(image_batch):
    features = []
    with torch.no_grad():
        for img in tqdm(image_batch):
            x = transform(img).unsqueeze(0).to(device)
            feat = feature_extractor(x).squeeze().cpu().numpy()
            features.append(feat)
    return np.stack(features)

# ✅ Твои данные:
# Пусть real_images, fake_images — это тензоры [N, 3, H, W]
# Они уже должны быть в [0, 1] или [-1, 1], преобразуются потом

X_real = extract_features(real_images)
X_fake = extract_features(fake_images)

# 4. Объединяем
X_all = np.vstack([X_real, X_fake])
y_all = np.array([1]*len(X_real) + [0]*len(X_fake))

# 5. Применяем TSNE
tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, random_state=42)
X_embedded = tsne.fit_transform(X_all)

# 6. Рисуем
plt.figure(figsize=(10, 8))
plt.scatter(X_embedded[y_all==1, 0], X_embedded[y_all==1, 1], label='Real', alpha=0.6)
plt.scatter(X_embedded[y_all==0, 0], X_embedded[y_all==0, 1], label='Fake', alpha=0.6)
plt.title("t-SNE визуализация распределения признаков")
plt.legend()
plt.grid(True)
plt.show()


Прокомментируйте получившийся результат: